# Requirements

- install node2vec code and add executable to `node2vec` directory in top level of the project (code: https://snap.stanford.edu/node2vec)
- compile GED code (graph embedding divergence), 
  the base implementation of the framework in C (the code is included, and can also be found at      https://github.com/ftheberge/Comparing_Graph_Embeddings) 
- new package to install:
```
using PyCall
run(`$(PyCall.python) -m pip install graphrole`)
```

In [ ]:
## the data directory
datadir="../Datasets/"

## location of the GED code
GED="../GED/GED"

## common colors
cls = ["red","green","blue"];

In [ ]:
using PyCall
using PyPlot
using LightGraphs
using GraphPlot
using Statistics
using Random
using LinearAlgebra
using DataFrames
using CSV
using DecisionTree
using FreqTables
using StatsBase
using Clustering

In [ ]:
ig = pyimport("igraph")
umap = pyimport("umap")
partition_igraph = pyimport("partition_igraph")
CHS = pyimport("sklearn.metrics").calinski_harabasz_score
AMI = pyimport("sklearn.metrics").adjusted_mutual_info_score
np = pyimport("numpy")
DBSCAN = pyimport("sklearn.cluster").dbscan
LogisticRegression = pyimport("sklearn.linear_model").LogisticRegression
roc_auc_score = pyimport("sklearn.metrics").roc_auc_score
roc_curve = pyimport("sklearn.metrics").roc_curve
npseed = pyimport("numpy.random").seed
npchoice = pyimport("numpy.random").choice
pickle_load = pyimport("pickle").load
RecursiveFeatureExtractor = pyimport("graphrole").RecursiveFeatureExtractor
RoleExtractor = pyimport("graphrole").RoleExtractor;

# A few useful functions

In [ ]:
function ig2lg(ig_g)
    lg_g = SimpleGraph(ig_g.vcount())
    for e in ig_g.es()
        add_edge!(lg_g, e.source + 1, e.target + 1)
    end
    return lg_g
end

In [ ]:
function binary_operator(u, v, op=:had)
    op == :had && return u .* v
    op == :l1 && return abs.(u .- v)
    op == :l2 && return (u .- v) .^ 2
    op == :avg && return (u .+ v) ./ 2.0
    throw(ArgumentError("unknown op"))
end

In [ ]:
function Hope(g, sim, dim; beta=0.01, alpha=0.5)
    dim = dim*2
    A = g.get_adjacency().data
    n = g.vcount()
    ## Katz
    if sim == :katz
        M_g = I - beta * A
        M_l = beta * A
    end
    ## Adamic-Adar
    if sim == :aa
        M_g = I
        D = diagm((x -> x > 1 ? 1/log(x) : 0.0).(g.degree()))
        M_l = A*D*A
        M_l[diagind(M_l)] .= 0.0
    end
    ## Common neighbors
    if sim == :cn
        M_g = I
        M_l = A*A
    end
    ## personalized page rank
    if sim == :ppr
        P = mapslices(A, dims=1) do x
            s = sum(x)
            iszero(s) ? fill(1/n, n) : x / s
        end
        M_g = I-alpha*P
        M_l = (1-alpha)*I
    end
    S = M_g \ M_l
    k = div(dim, 2)
    u, s, vt = svd(S)
    X1 = u[:, 1:k] * diagm(sqrt.(s[1:k]))
    ## undirected graphs have identical source and target embeddings
    if !g.is_directed()
        return X1
    else
        X2 = vtu[:, 1:k] * diagm(sqrt.(s[1:k]))
        return [X1 X2]
    end
end

In [ ]:
## save to disk to compute divergence
function saveEmbedding(X, g, fn="_embed")
    names = g.vs.get_attribute_values("name")
    open(fn, "w") do f
        println(f, size(X,1), " ", size(X, 2))
        for i in axes(X, 1)
            print(f, names[i], " ")
            for j in axes(X, 2)
                print(f, X[i, j])
                 j < size(X, 2) && print(f, " ")
            end
            println(f)
        end
    end
end

In [ ]:
## Computing JS divergence with GED code given edgelist, communities and embedding
function JS(edge_file, comm_file, embed_file, entropy=false)
    if entropy
        x = `$GED -E -g $edge_file -c $comm_file -e $embed_file`
    else
        x = `$GED -g $edge_file -c $comm_file -e $embed_file`
    end
    io = IOBuffer()
    run(pipeline(x, stdout=io), wait=true)
    s = split(String(take!(io)), " ")
    return parse(Float64, s[2])
end

In [ ]:
## Laplacian eigenmaps embedding
function LE(g, dim=2)
    L_sym = g.laplacian(normalized=true)
    w, v = eigen(L_sym)
    return real.(v[:, 2:dim+1])
end


In [ ]:
## Read embedding from file in node2vec format
## Map to layout format
## for visualization, we use UMAP if dim > 2
function embed2layout(fn="_embed"; graph)
    D = CSV.read(fn, DataFrame, header=false, datarow=2)
    if eltype(D[!, end]) === Missing
        D = D[!, 1:end-1]
    end
    
    df1 = DataFrame(Column1=parse.(Int, graph.vs.get_attribute_values("name")))
    df1.id = axes(df1, 1)
    df2 = leftjoin(df1, D, on=:Column1)
    disallowmissing!(sort!(df2, :id))
    
    Y = Matrix(select(df2, Not(1:2)))
    if size(Y,2) >  2
        Y = umap.UMAP().fit_transform(Y)
    end
    return Y
end

In [ ]:
function readEmbedding(fn="_embed"; N2K=nothing)
    D = CSV.read(fn, DataFrame, header=false, datarow=2)
    if eltype(D[!, end]) === Missing
        D = D[!, 1:end-1]
    end
    Y = Matrix(select(D, Not(1)))

    if N2K !== nothing
        x = [N2K[i] for i in D[:, 1]]
        Y = Y[sortperm(x), :]
    end
    return Y
end

# Load and prepare datasets

* g: small ABCD graph (100 nodes), mainly for visualization and quick exampes
* G: larger ABCD graph (1000 nodes), for experiments
* z: zachary graph, for visualzation

## 1. Small ABCD graph 

In [ ]:
## read graph and communities
g = ig.Graph.Read_Ncol(datadir * "ABCD/abcd_100.dat", directed=false)
c = np.loadtxt(datadir * "ABCD/abcd_100_comms.dat", dtype="uint16", usecols=(1))
g.vs.set_attribute_values("comm", [c[parse(Int, x.attributes()["name"])] for x in g.vs])

## print a few stats
println(g.vcount()," vertices, ",
        g.ecount()," edges; ",
        "avg degreee: ", mean(g.degree()),
        ", communities: ",maximum(g.vs.get_attribute_values("comm")))

## ground truth
gt = Dict(enumerate(g.vs.get_attribute_values("comm")))
## map between name to key
n2k = Dict((v, k) for (k, v) in enumerate(g.vs.get_attribute_values("name")))

v_color = cls[g.vs.get_attribute_values("comm")]
g_lg = ig2lg(g)
Random.seed!(2)
gplot(g_lg,
      NODESIZE=0.03, nodefillc=v_color,
      EDGELINEWIDTH=0.2, edgestrokec="gray")

## 2. Larger ABCD graph

In [ ]:
## read graph and communities
G = ig.Graph.Read_Ncol(datadir * "ABCD/abcd_1000.dat", directed=false)
C = np.loadtxt(datadir * "ABCD/abcd_1000_comms.dat", dtype="uint16", usecols=(1))
G.vs.set_attribute_values("comm", [C[parse(Int, x.attributes()["name"])] for x in G.vs])

## print a few stats
println(G.vcount()," vertices, ",
        G.ecount()," edges; ",
        "avg degreee: ", mean(G.degree()),
        ", communities: ",maximum(G.vs.get_attribute_values("comm")))

## ground truth
GT = Dict(enumerate(G.vs.get_attribute_values("comm")))
## map between name to key
N2K = Dict((v, k) for (k, v) in enumerate(G.vs.get_attribute_values("name")))

G_LG = ig2lg(G)
Random.seed!(2)
gplot(G_LG,
      NODESIZE=0.01, nodefillc="black",
      EDGELINEWIDTH=0.1, edgestrokec="gray") ## communities are far from obvious in 2d layout!

## 3. Zachary (karate) graph

In [ ]:
z = ig.Graph.Famous("zachary")
c = [0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1] .+ 1
z.vs.set_attribute_values("name", string.(0:z.vcount()-1))
z.vs.set_attribute_values("comm", [c[parse(Int, x.attributes()["name"])+1] for x in z.vs])

v_color = cls[z.vs.get_attribute_values("comm")]
z_lg = ig2lg(z)
Random.seed!(2)
gplot(z_lg,
      NODESIZE=0.03, nodefillc=v_color,
      EDGELINEWIDTH=0.2, edgestrokec="gray")

# Show various 2d layouts using small Zachary graph

In [ ]:
gplot(z_lg, layout=random_layout,
      NODESIZE=0.03, nodefillc=v_color,
      EDGELINEWIDTH=0.2, edgestrokec="gray")

In [ ]:
gplot(z_lg, layout=circular_layout,
      NODESIZE=0.03, nodefillc=v_color,
      EDGELINEWIDTH=0.2, edgestrokec="gray")

In [ ]:
gplot(z_lg, layout=spectral_layout,
      NODESIZE=0.03, nodefillc=v_color,
      EDGELINEWIDTH=0.2, edgestrokec="gray")

# Perform several embeddings -- Zachary graph
* node2vec from source code
* HOPE with different similarities
* Laplacian Eigenmaps
* visualize some good and bad results

We use the framework to compute the "graph embedding divergence" (GED.c)

In [ ]:
L=DataFrame()
DIM=[5,10,15]

best_jsd = 1.1
worst_jsd = -0.1

## Hope
for dim in DIM, sim in (:katz,:ppr, :cn, :aa)
    X = Hope(z,sim,dim)
    saveEmbedding(X,z)
    jsd = JS(datadir * "Zachary/zachary.edgelist", datadir * "Zachary/zachary.ecg", "_embed")
    
    if jsd < best_jsd
        cp("_embed", "_embed_best", force=true)
        global best_jsd = jsd
    end
    if jsd > worst_jsd
        cp("_embed", "_embed_worst", force=true)
        global worst_jsd = jsd
    end

    push!(L, (dim=dim, alg="hope", param=sim, jsd=jsd))
end

## LE
for dim in DIM
    X = LE(z,dim)
    saveEmbedding(X,z)
    jsd = JS(datadir * "Zachary/zachary.edgelist", datadir * "Zachary/zachary.ecg", "_embed")

    if jsd < best_jsd
        cp("_embed", "_embed_best", force=true)
        global best_jsd = jsd
    end
    if jsd > worst_jsd
        cp("_embed", "_embed_worst", force=true)
        global worst_jsd = jsd
    end

    push!(L, (dim=dim, alg="le", param=nothing,jsd=jsd), cols=:union)
end

## node2vec is in my path
for dim in DIM, (p,q) in [(1,0.1),(0.1,1),(1,1)]
        x = `../node2vec/node2vec -i:$datadir/Zachary/zachary.edgelist -o:_embed -d:$dim -p:$p -q:$q -l:15`
        run(x, wait=true)
        jsd = JS(datadir * "Zachary/zachary.edgelist" ,datadir * "Zachary/zachary.ecg", "_embed")

        if jsd < best_jsd
            cp("_embed", "_embed_best", force=true)
            global best_jsd = jsd
        end
        if jsd > worst_jsd
            cp("_embed", "_embed_worst", force=true)
            global worst_jsd = jsd
        end

        push!(L, (dim=dim, alg="n2v", param=(p,q),jsd=jsd), cols=:union)
end

In [ ]:
sort!(L, :jsd);
first(L, 10)

In [ ]:
## plot top result
l = embed2layout("_embed_best", graph=z)
gplot(z_lg, l[:, 1], l[:, 2],
      NODESIZE=0.03, nodefillc=v_color,
      EDGELINEWIDTH=0.2, edgestrokec="gray")

In [ ]:
last(L, 10)

In [ ]:
## plot bottom result

l = embed2layout("_embed_worst", graph=z)
gplot(z_lg, l[:, 1], l[:, 2],
      NODESIZE=0.03, nodefillc=v_color,
      EDGELINEWIDTH=0.2, edgestrokec="gray")

# Perform several embeddings -- small ABCD  graph
* node2vec from source code
* HOPE different similarities
* Laplacian Eigenmaps
* visualize some good and bad results

In [ ]:
L = DataFrame()
DIM = [2,4,8,16,24,32]

best_jsd = 1.1
worst_jsd = -0.1

## Hope
for dim in DIM, sim in (:katz,:ppr, :cn, :aa)
    X = Hope(g,sim,dim)
    saveEmbedding(X,g)
    jsd = JS(datadir * "ABCD/abcd_100.dat", datadir * "ABCD/abcd_100.ecg", "_embed")
    
    if jsd < best_jsd
        cp("_embed", "_embed_best", force=true)
        global best_jsd = jsd
    end
    if jsd > worst_jsd
        cp("_embed", "_embed_worst", force=true)
        global worst_jsd = jsd
    end

    push!(L, (dim=dim, alg="hope", param=sim, jsd=jsd))
end

## LE
for dim in DIM
    X = LE(g,dim)
    saveEmbedding(X,g)
    jsd = JS(datadir * "ABCD/abcd_100.dat", datadir * "ABCD/abcd_100.ecg", "_embed")
    
    if jsd < best_jsd
        cp("_embed", "_embed_best", force=true)
        global best_jsd = jsd
    end
    if jsd > worst_jsd
        cp("_embed", "_embed_worst", force=true)
        global worst_jsd = jsd
    end

    push!(L, (dim=dim, alg="le", param=nothing,jsd=jsd), cols=:union)
end

## node2vec is in my path
for dim in DIM, (p,q) in [(1,0.1),(0.1,1),(1,1)]
        x = `../node2vec/node2vec -i:$datadir/ABCD/abcd_100.dat -o:_embed -d:$dim -p:$p -q:$q -l:15 -l:15`
        run(x, wait=true)
        jsd = JS(datadir * "ABCD/abcd_100.dat" ,datadir * "ABCD/abcd_100.ecg", "_embed")
    
        if jsd < best_jsd
            cp("_embed", "_embed_best", force=true)
            global best_jsd = jsd
        end
        if jsd > worst_jsd
        cp("_embed", "_embed_worst", force=true)
            global worst_jsd = jsd
        end

        push!(L, (dim=dim, alg="n2v", param=(p,q),jsd=jsd), cols=:union)
end

In [ ]:
sort!(L, :jsd)
first(L, 10)

In [ ]:
## re-run and plot top result
l = embed2layout("_embed_best", graph=g)
v_color = cls[g.vs.get_attribute_values("comm")]
gplot(g_lg, l[:, 1], l[:, 2],
      NODESIZE=0.03, nodefillc=v_color,
      EDGELINEWIDTH=0.2, edgestrokec="gray")

In [ ]:
last(L, 10)

In [ ]:
## plot bottom result
l = embed2layout("_embed_worst", graph=g)
 gplot(g_lg, l[:, 1], l[:, 2],
       NODESIZE=0.03, nodefillc=v_color,
       EDGELINEWIDTH=0.2, edgestrokec="gray")

# Large ABCD graph -- find a good embedding with the framework
* we only look as 16 configurations with HOPE for now (for speed)
* we'll consider more in the large classification experiment later

The good embedding was generated using the code provided in the Python notebook

# Classification on larger ABCD graph

* we use a good embedding saved from above cell
* we use a random forest model on embedded space
* we split the data as train and test
* the goal is to recover the communities for each node


In [ ]:
## used saved "best" embedding from above
X = readEmbedding(datadir * "ABCD/abcd_1000_embed_best")
y = string.(G.vs.get_attribute_values("comm"))
## train/test split
Random.seed!(1234)
to_train = falses(length(y))
to_train[1:round(Int, length(y)*0.25)] .= true
shuffle!(to_train)

X_train = X[to_train, :]
X_test = X[.!to_train, :]
y_train = y[to_train]
y_test = y[.!to_train]

model = build_forest(y_train, X_train, size(X, 2) ÷ 2, 100, 0.5)

y_pred = apply_forest(model, X_test)

cm = freqtable(y_test, y_pred)

In [ ]:
println("accuracy: $(sum(diag(cm))/length(y_test))")

In [ ]:
## compare with random classifier -- assuming we know only the number of classes (12)
describe([sum(diag(freqtable(rand(1:12, length(y_test)), y_test)))/length(y_test) for i in 1:10000])

In [ ]:
## compare with random classifier -- using class proportions in training data
describe([sum(diag(freqtable(rand(y_train, length(y_test)), y_test)))/length(y_test) for i in 1:10000])

# Clustering
* we run graph clustering (Louvain, ECG)
* we compare with vector space embedding using same embedding
* we use k-means (various k) and DBSCAN
* recall there are 12 ground truth community

In [ ]:
## again we use 'good' embedding from before
X = readEmbedding(datadir * "ABCD/abcd_1000_embed_best")

L = DataFrame()
K = [6,9,12,15,24] ## for k-means (real number of clusters is 12)
REP = 30

for i in 1:REP
    ## kmeans
    for k in K
        labels = kmeans(X', k).assignments
        scr = CHS(X, labels)
        ami = AMI(G.vs.get_attribute_values("comm"), labels)
        push!(L, (method="km", param=k, scr=scr, ami=ami))
    end
    ## ECG
    ec = G.community_ecg().membership
    scr = G.modularity(ec)
    ami = AMI(G.vs.get_attribute_values("comm"),ec)
    push!(L, (method="ecg", param=nothing, scr=scr, ami=ami), cols=:union)
    ## Louvain -- permute as this is not done in igraph
    p = randperm(G.vcount()) .- 1
    GG = G.permute_vertices(Any[x for x in p])
    l = GG.community_multilevel().membership
    ll = similar(l)
    for i in 1:length(l)
        ll[i] = l[p[i] + 1]
    end
    scr = G.modularity(ll)
    ami = AMI(G.vs.get_attribute_values("comm"),ll)
    push!(L, (method="ml", param=nothing, scr=scr, ami=ami), cols=:union)
end

combine(groupby(L, :method), x -> last(sort(x, :scr)))

In [ ]:
gL = groupby(L, [:method, :param], sort=true)

boxplot([sdf.ami for sdf in gL])
xticks(1:7, [isnothing(p) ? m : "$m($p)" for (m,p) in keys(gL)]);

In [ ]:
## DBSCAN -- we tried different epsilon and dim
## test via calinski_harabasz_score (CHS) or silhouette_score or davies_bouldin_score
## best result obtained empirically with min_samples = 8

top = 0
for dim in [4,8,16,24,32,40,48,64], ms in [8]
    U = umap.UMAP(n_components=24).fit_transform(X)
    for e in 0.4:0.0025:0.5
        cl = DBSCAN(U, eps=e, min_samples=ms)
        labels = cl[2]
        s = CHS(U,labels) ## score
        if s > top
            global top = s
            global e_top = e
            global d_top = dim
            global m_top = ms
        end
    end
end

U = umap.UMAP(n_components=d_top).fit_transform(X)
cl = DBSCAN(U, eps=e_top, min_samples=m_top)
labels = cl[2]

b = labels .> -1
println("AMI without outliers: ", AMI(G.vs.get_attribute_values("comm")[b], labels[b]))
println("AMI with outliers: ", AMI(G.vs.get_attribute_values("comm"), labels))

# Link prediction

* we drop 10% edges and re-compute the embedding
* we train a logistic regression model
* we apply final model to test set

## First try with noisy graph

Recall that xi=0.6, the proportion of noise edges


In [ ]:
## pick 10% edges at random, save new graph as Gp
Random.seed!(12345)
test_size = round(Int, 0.1*G.ecount())

npseed(123456)
test_eid = npchoice(G.ecount(),size=test_size,replace=false)

#test_eid = sample(0:G.ecount()-1, test_size, replace=false)
Gp = G.copy()
Gp.delete_edges(test_eid)

## compute embedding on Gp 
X = Hope(Gp, :ppr, 48);

In [ ]:
## Train model with Hadamard binary operator (other choices are 'l1', 'l2 and 'avg')
op = :had

## Build training data, first the edges
F = [binary_operator(X[e.tuple[1]+1, :],X[e.tuple[2]+1, :],op) for e in Gp.es]
sz = length(F)
f = [fill(1, sz); fill(0, sz)]

## then for equal number of non-edges (we over-sample to drop edges and collisions from the list)
e = [minmax(rand(0:Gp.vcount()-1), rand(0:Gp.vcount()-1)) for i in 1:2*sz]
unique!(e) # drop collisions
filter!(((a, b),) -> Gp.get_eid(a, b, directed=false, error=false) == -1, e)
non_edges = e[1:sz]
## features for node pairs without edges
for (a, b) in non_edges
    push!(F, binary_operator(X[a+1, :],X[b+1, :], op))
end

## train model
logreg = LogisticRegression()
logreg.fit(F, f)

## prepare test set, first with all dropped edges from G 
X_test = [(e = G.es[i+1]; binary_operator(X[e.tuple[1]+1, :],X[e.tuple[2]+1, :],op)) for i in test_eid]
szt = length(X_test)
y_test = [fill(1, szt); fill(0, szt)]

## then for equal number of non-edges (we over-sample to drop edges and collisions from the list)
e2 = [minmax(rand(0:G.vcount()-1), rand(0:G.vcount()-1)) for i in 1:2*szt]
unique!(e2) # drop collisions
filter!(((a, b),) -> G.get_eid(a, b, directed=false, error=false) == -1, e2)
non_edges2 = e2[1:szt]
for (a, b) in non_edges2
    push!(X_test, binary_operator(X[a+1, :],X[b+1, :], op))
end

## apply the model to test data
println("Accuracy of logistic regression classifier with $op on test set: ", logreg.score(X_test, y_test))
print("AUC: ",roc_auc_score(y_test, logreg.predict_proba(X_test)[:,2]))

## Link prediction with less noisy graph

In [ ]:
## read graph and communities - graph with xi=0.2
G2 = ig.Graph.Read_Ncol(datadir * "ABCD/abcd_1000_xi2.dat", directed=false)
C = np.loadtxt(datadir * "ABCD/abcd_1000_xi2_comms.dat", dtype="uint16", usecols=(1))
G.vs.set_attribute_values("comm", [C[parse(Int, x.attributes()["name"])] for x in G.vs])

## pick 10% edges at random, save new graph as Gp
Random.seed!(12345)
test_size = round(Int, 0.2*G2.ecount())

npseed(123456)
test_eid = npchoice(G2.ecount(),size=test_size,replace=false)

#test_eid = sample(0:G.ecount()-1, test_size, replace=false)
Gp = G2.copy()
Gp.delete_edges(test_eid)

## compute embedding on Gp 
X = Hope(Gp, :ppr, 48);

In [ ]:
## Train model with Hadamard binary operator (other choices are 'l1', 'l2 and 'avg')
op = :had

## Build training data, first the edges
F = [binary_operator(X[e.tuple[1]+1, :],X[e.tuple[2]+1, :],op) for e in Gp.es]
sz = length(F)
f = [fill(1, sz); fill(0, sz)]

## then for equal number of non-edges (we over-sample to drop edges and collisions from the list)
e = [minmax(rand(0:Gp.vcount()-1), rand(0:Gp.vcount()-1)) for i in 1:2*sz]
unique!(e) # drop collisions
filter!(((a, b),) -> Gp.get_eid(a, b, directed=false, error=false) == -1, e)
non_edges = e[1:sz]
## features for node pairs without edges
for (a, b) in non_edges
    push!(F, binary_operator(X[a+1, :],X[b+1, :], op))
end

## train model
logreg = LogisticRegression()
logreg.fit(F, f)

## prepare test set, first with all dropped edges from G 
X_test = [(e = G2.es[i+1]; binary_operator(X[e.tuple[1]+1, :],X[e.tuple[2]+1, :],op)) for i in test_eid]
szt = length(X_test)
y_test = [fill(1, szt); fill(0, szt)]

## then for equal number of non-edges (we over-sample to drop edges and collisions from the list)
e2 = [minmax(rand(0:G2.vcount()-1), rand(0:G2.vcount()-1)) for i in 1:2*szt]
unique!(e2) # drop collisions
filter!(((a, b),) -> G2.get_eid(a, b, directed=false, error=false) == -1, e2)
non_edges2 = e2[1:szt]
for (a, b) in non_edges2
    push!(X_test, binary_operator(X[a+1, :],X[b+1, :], op))
end

## apply the model to test data
println("Accuracy of logistic regression classifier with $op on test set: ", logreg.score(X_test, y_test))
print("AUC: ",roc_auc_score(y_test, logreg.predict_proba(X_test)[:,2]))

In [ ]:
logit_roc_auc = round(10000*roc_auc_score(y_test, logreg.predict_proba(X_test)[:,2]))/100
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,2])
plot(fpr, tpr, color="gray",label="Logistic Regression (AUC = $logit_roc_auc%)")
plot([0, 1], [0, 1],"k--")
xlim([0.0, 1.0])
ylim([0.0, 1.05])
xlabel("False Positive Rate")
ylabel("True Positive Rate")
title("")
legend(loc="lower right");

## Larger study -- use accuracy for picking embedding

- we training-validation-test split
- this can be long to run -- a pickle file with the results is included in data directory
- to re-run from scratch please check Python notebooks

In [ ]:
## load L and train/val/test ids
fh = py"open"(datadir * "ABCD/abcd_1000_embeddings.pkl", "rb")
id_train,id_val,id_trainval,id_test,L = pickle_load(fh)
fh.close()
y_all = G.vs.get_attribute_values("comm")
y_train = [y_all[i+1] for i in id_train]
y_trainval = [y_all[i+1] for i in id_trainval]
y_val = [y_all[i+1] for i in id_val]
y_test = [y_all[i+1] for i in id_test];

In [ ]:
R = identity.(DataFrame(L,[:dim,:algo,:param,:div,:acc]))
print("Kendall's correlation: ", corkendall(R.div, R.acc))

In [ ]:
## sort by Divergence on validation set
sort!(R, :div)
R.rank_div = axes(R, 1)
first(R, 5)

In [ ]:
## sort by Accuracy on validation set
sort!(R, :acc, rev=true)
R.rank_acc = axes(R, 1)
first(R, 5)

In [ ]:
## quite a range of accuracy on the validation set!
last(R, 5)

##  Apply to test set. 

This takes several minutes to run so a pickle file is provided.

Source codes for generating the data are provided in Python notebooks.

In [ ]:
## load L and train/val/test ids
fh = py"open"(datadir * "ABCD/abcd_1000_embeddings_test.pkl", "rb")
R.test = pickle_load(fh)
fh.close()
println("mean accuracy over all models on test set:",mean(R.test))

In [ ]:
## sort by Accuracy on validation set
sort!(R, :test, rev=true)
R.rank_test = axes(R, 1)
first(R, 5)

In [ ]:
## top results on test set w.r.t. divergence on validation set
sort!(R, :div)
top_div = R.test[1:10]

## top results on test set w.r.t. accuracy on validation set
sort!(R, :acc, rev=true)
top_acc = R.test[1:10]

In [ ]:
boxplot([top_acc, top_div])
xticks(1:2, ["Top-10 validation set accuracy","Top-10 divergence score"]);

In [ ]:
plot(R.rank_acc,R.test,".",color="black")
xlabel("Rank",fontsize=14)
ylabel("Test set accuracy",fontsize=14);

In [ ]:
plot(R.rank_div,R.test,".",color="black")
xlabel("Rank",fontsize=14)
ylabel("Test set accuracy",fontsize=14);

In [ ]:
print((cor(R.rank_acc,R.test),
       cor(R.rank_div,R.test)))

In [ ]:
function sample_random_classifier()
    y_pred = sample(1:12, Weights(proptable(y_trainval)), length(y_test))
    return sum(diag(proptable(y_test, y_pred)))
end
println("Random classifier accuracy on test set: ")
describe([sample_random_classifier() for i in 1:10000])

## ReFex: illustrate roles on Zachary graph

We use the 'graphrole' package


In [ ]:
# extract features
feature_extractor = RecursiveFeatureExtractor(z, max_generations=5)
features = feature_extractor.extract_features()
println("Features extracted from $(feature_extractor.generation_count) recursive generations:")
features.head(10)

In [ ]:
# assign node roles in a dictionary
role_extractor = RoleExtractor(n_roles=3)
role_extractor.extract_role_factors(features)
node_roles = role_extractor.roles
role_extractor.role_percentage.head()

In [ ]:
unique_roles = sort(unique(values(node_roles)))
cls = ["red","blue","green"]
# map roles to colors
role_colors = Dict(unique_roles .=> cls)
v_color = [role_colors[node_roles[i]] for i in 0:z.vcount()-1]
Random.seed!(2)
gplot(z_lg,
      NODESIZE=0.03, nodefillc=v_color,
      EDGELINEWIDTH=0.2, edgestrokec="gray")

# Anomaly detection

## Dataset -- American College Football Graph

[REF]: "Community structure in social and biological networks", M. Girvan and M. E. J. Newman
PNAS June 11, 2002 99 (12) 7821-7826; https://doi.org/10.1073/pnas.122653799


Teams are part of 12 conferences (the 'communities'):
*   0 = Atlantic Coast
*   1 = Big East
*   2 = Big Ten
*   3 = Big Twelve
*   4 = Conference USA
*   5 = Independents
*   6 = Mid-American
*   7 = Mountain West
*   8 = Pacific Ten
*   9 = Southeastern
*  10 = Sun Belt
*  11 = Western Athletic

14 teams out of 115 appear as anomalies as can be seen in Figure 5 of [REF], namely:
- 5 teams in #5 conference (Independent) play teams in other conferences (green triangles)
- 7 teams in #10 conference (Sun Belt) are broken in 2 clumps (pink triangles) 
- 2 teams from #11 conference play mainly with #10 conference (red triangles)

Here, we try to recover those anomalous teams by running several embeddings (we use node2vec):

- for each embedding:
 - compute divergence using our framework
 - also compute entropy of b-vector for each node (probability distribution of edges w.r.t. every community in the geometric Chung-Lu model)
- plot entropy vs divergence
- for some good/bad embedding, boxplot entropy of anomalous vs other nodes



In [ ]:
comm_color = ["red", "red", "green", "green", "blue", "blue",
              "black", "black", "yellow", "yellow", "orange", "orange"]
comm_label = repeat(["", "x"], 6)

## read graph and communities
g = ig.Graph.Read_Ncol(datadir * "Football/football.edgelist", directed=false)
c = parse.(Int, readlines(datadir * "Football/football.community"))
g.vs.set_attribute_values("community", [c[1 + parse(Int, x.attributes()["name"])] for x in g.vs])

anomaly = Int[c in [5, 10] || n in ["28", "58"] for
              (c, n) in zip(g.vs.get_attribute_values("community"), g.vs.get_attribute_values("name"))]

# communities are marked with color and an optional "x" mark on the node
# if the mark is "A" and node is a bit larger it is an anomaly
Random.seed!(1234)
lab = comm_label[g.vs.get_attribute_values("community") .+ 1]
lab[Bool.(anomaly)] .= "A"
gplot(ig2lg(g),
      NODESIZE=0.03 .+ anomaly ./75, nodefillc=comm_color[g.vs.get_attribute_values("community") .+ 1],
      nodelabel=lab,
      nodelabelc="white",
      EDGELINEWIDTH=0.2, edgestrokec="gray")

In [ ]:
best_jsd = 1.1
worst_jsd = -0.1
## node2vec with varying parameters (60 embeddings)
L = DataFrame()
for dim in 2:2:24
    for (p,q) in [(1.0, 0.5),(0.5, 1.0),(1.0, 0.01),(0.01, 1.0),(1.0, 1.0)]
        x = `../node2vec/node2vec -i:$datadir/Football/football.edgelist -o:_embed -d:$dim -p:$p -q:$q`
        run(x, wait=true)
        jsd = JS(datadir * "Football/football.edgelist" ,datadir * "Football/football.ecg", "_embed", true)
        if jsd < best_jsd
            cp("_entropy", "_entropy_best", force=true)
            global best_jsd = jsd
        end
        if jsd > worst_jsd
            cp("_entropy", "_entropy_worst", force=true)
            global worst_jsd = jsd
        end
        ent = parse.(Float64, getindex.(split.(readlines("_entropy"), ','), 2))
        roc = roc_auc_score(anomaly, ent)
        push!(L, (dim=dim, algo="n2v", param=(p,q),jsd=jsd, auc=roc))
    end
end

In [ ]:
sort!(L, :jsd)
first(L, 5)

In [ ]:
last(L, 5)

In [ ]:
## auc vs divergence (jsd)
plot(L.jsd,L.auc,"o",color="black")
xlabel("JS Divergence",fontsize=14)
ylabel("AUC",fontsize=14);

In [ ]:
## Entropy scores - some good embedding
ent = parse.(Float64, getindex.(split.(readlines("_entropy_best"), ','), 2))
boxplot([ent[anomaly .== 0], ent[anomaly .== 1]],
        labels=["Regular", "Anomalous"], sym=".",whis=(0,100), widths=.5)
title("Low divergence embedding",fontsize=14)
ylabel("Entropy",fontsize=14);

In [ ]:
## Entropy scores - not so good embedding
ent = parse.(Float64, getindex.(split.(readlines("_entropy_worst"), ','), 2))
boxplot([ent[anomaly .== 0], ent[anomaly .== 1]],
             labels=["Regular", "Anomalous"], sym=".",whis=(0,100), widths=.5)
title("Low divergence embedding",fontsize=14)
ylabel("Entropy",fontsize=14);

## AUC using average rank with several top embeddings

In [ ]:
k = 7
ranking = fill(0.0, g.vcount())
for i in 1:k
    dim = L.dim[i]
    p,q = L.param[i]
    x = `../node2vec/node2vec -i:$datadir/Football/football.edgelist -o:_embed -d:$dim -p:$p -q:$q`
    run(x, wait=true)
    jsd = JS(datadir * "Football/football.edgelist" ,datadir * "Football/football.ecg", "_embed", true)
    ent = parse.(Float64, getindex.(split.(readlines("_entropy"), ','), 2))
    global ranking .+= ordinalrank(ent)
end

In [ ]:
println("AUC: ",roc_auc_score(anomaly, ranking))